#  Factoring Lab

In [1]:
from vec import Vec
from vecutil import list2vec
from GF2 import one

from factoring_support import dumb_factor
from factoring_support import intsqrt
from factoring_support import gcd
from factoring_support import primes
from factoring_support import prod

import echelon
import random

gcd(m,n) return the greatest common divisor of positive integers m and n. This
algorithm is attributed to Euclid, and it is very fast.

In [2]:
def gcd(x,y): return x if y == 0 else gcd(y, x % y)

In [3]:
gcd(276534813447635747652, 333070702552660863114)

18172055646

The module factoring support provides a procedure intsqrt(x) with the following spec:

• input: an integer x

• output: an integer y such that y ∗ y is close to x and, if x happens to be a perfect square, y ∗ y is
exactly x.


In [3]:
def root_method(N):
    a = float(intsqrt(N)+1)
    logic = False
    while not logic:
        if a.is_integer():
            b = intsqrt(a**2-N)
            logic = True
        else:
            a+=1
    return a-b

In [8]:
root_method(55)

5.0

In [9]:
root_method(77)

5.0

In [10]:
root_method(146771)

358.0

In [11]:
root_method(118)

7.0

In [27]:
r = random.randint(100000, 1000000)
s = random.randint(100000, 1000000)
t = random.randint(100000, 1000000)

In [30]:
a = r*s; b = s*t
# greatest common divisor
d = gcd(a, b)

In [32]:
# a is divisible by b
a%d == 0

True

In [33]:
# b is divisible by d
b%d ==0

True

In [34]:
d>=s

True

In [35]:
N = 367160330145890434494322103
a = 67469780066325164
b = 9429601150488992

In [36]:
# verify that a ∗ a − b ∗ b is divisible by N
(a**2 - b**2)%N == 0

True

That means that the greatest common
divisor of a − b and N has a chance of being a nontrivial divisor of N . Test this using the gcd procedure,
and report the nontrivial divisor you found.


In [38]:
gcd(a-b, N)

19117318483477

How can we find such a pair of integers? Instead of hoping to get lucky, we’ll take matters into our
own hands. We’ll try to create a and b. This method starts by creating a set primeset consisting of the
first thousand or so primes. We say an integer x factors over primeset if you can multiply together some of
the primes in S (possibly using a prime more than once) to form x.


The module factoring support defines a procedure dumb factor(x, primeset) with the following spec:

• input: an integer x and a set primeset of primes

• output: if there are primes p1 , . . . , ps in primeset and positive integers e1 , e2, . . . , es (the exponents) such that x = pe 1 1 pe 2 2 · · · pe ss then the procedure returns the list [(p1, e1 ), (p2 , e2 ), . . . , (ps , es )] of pairs (prime, exponent). If not, the procedure returns the empty list.


In [39]:
dumb_factor(75, {2,3,5,7})

[(3, 1), (5, 2)]

In [40]:
dumb_factor(30, {2,3,5,7})

[(2, 1), (3, 1), (5, 1)]

In [41]:
dumb_factor(1176, {2,3,5,7})

[(2, 3), (3, 1), (7, 2)]

In [42]:
 dumb_factor(2*17, {2,3,5,7})

[]

In [43]:
dumb_factor(2*3*5*19, {2,3,5,7})

[]

In [47]:
primeset={2, 3, 5, 7, 11, 13}
x = [12, 154, 2 * 3 * 3 * 3 * 11 * 11 * 13, 2 * 17, 2 * 3 * 5 * 7 * 19]
for number in x:
    print (dumb_factor(number, x))

[(12, 1)]
[(154, 1)]
[(84942, 1)]
[(34, 1)]
[(3990, 1)]


In [4]:
## Task 1
def int2GF2(i):
    '''
    Returns one if i is odd, 0 otherwise.

    Input:
        - i: an int
    Output:
        - one if i is congruent to 1 mod 2
        - 0   if i is congruent to 0 mod 2
    Examples:
        >>> int2GF2(3)
        one
        >>> int2GF2(100)
        0
    '''
    return 0 if i%2 == 0 else one

In [49]:
int2GF2(3)

one

In [50]:
int2GF2(100)

0

In [5]:
## Task 2
def make_Vec(primeset, factors):
    '''
    Input:
        - primeset: a set of primes
        - factors: a list of factors [(p_1,a_1), ..., (p_n, a_n)]
                   with p_i in primeset
    Output:
        - a vector v over GF(2) with domain primeset
          such that v[p_i] = int2GF2(a_i) for all i
    Example:
        >>> make_Vec({2,3,11}, [(2,3), (3,2)]) == Vec({2,3,11},{2:one})
        True
    '''
    return Vec(primeset, {pair[0]:int2GF2(pair[1]) for pair in factors})

In [56]:
make_Vec({2,3,11}, [(2,3), (3,2)]) == Vec({2,3,11},{2:one})

True

In [57]:
make_Vec({2,3,5,7,11}, [(2,17), (3, 0), (5,1), (11,3)])

Vec({2, 11, 3, 5, 7},{11: one, 2: one, 3: 0, 5: one})

In [6]:
## Task 3
def find_candidates(N, primeset):
    '''
    Input:
        - N: an int to factor
        - primeset: a set of primes

    Output:
        - a tuple (roots, rowlist)
        - roots: a list a_0, a_1, ..., a_n where a_i*a_i - N can be factored
                 over primeset
        - rowlist: a list such that rowlist[i] is a
                   primeset-vector over GF(2) corresponding to a_i
          such that len(roots) = len(rowlist) and len(roots) > len(primeset)
    Example:
        >>> from factoring_support import primes
        >>> N = 2419
        >>> primeset = primes(32)
        >>> roots, rowlist = find_candidates(N, primeset)
        >>> set(roots) == set([51, 52, 53, 58, 61, 62, 63, 67, 68, 71, 77, 79])
        True
        >>> D = {2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31}
        >>> set(rowlist) == set([Vec(D,{2: one, 13: one, 7: one}),\
                Vec(D,{3: one, 19: one, 5: one}),\
                Vec(D,{2: one, 3: one, 5: one, 13: one}),\
                Vec(D,{3: one, 5: one, 7: one}),\
                Vec(D,{7: one, 2: one, 3: one, 31: one}),\
                Vec(D,{3: one, 19: one}),\
                Vec(D,{2: one, 31: one}),\
                Vec(D,{2: one, 5: one, 23: one}),\
                Vec(D,{5: one}),\
                Vec(D,{3: one, 2: one, 19: one, 23: one}),\
                Vec(D,{2: one, 3: one, 5: one, 13: one}),\
                Vec(D,{2: one, 3: one, 13: one})])
        True
    '''
    roots = []
    rowlist = []
    x = intsqrt(N)+2
    
    while len(roots) < len(primeset)+1:
        d = lambda x: dumb_factor(x**2-N, primeset)
        if len (d(x)) != 0:
            roots.append(x)
            rowlist.append(make_Vec(primeset, d(x)))
        x+=1
    return (roots, rowlist)

In [77]:
from factoring_support import primes
N = 2419
primeset = primes(32)
roots, rowlist = find_candidates(N, primeset)
set(roots) == set([51, 52, 53, 58, 61, 62, 63, 67, 68, 71, 77, 79])

True

In [78]:
D = {2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31}
set(rowlist) == set([Vec(D,{2: one, 13: one, 7: one}),\
                Vec(D,{3: one, 19: one, 5: one}),\
                Vec(D,{2: one, 3: one, 5: one, 13: one}),\
                Vec(D,{3: one, 5: one, 7: one}),\
                Vec(D,{7: one, 2: one, 3: one, 31: one}),\
                Vec(D,{3: one, 19: one}),\
                Vec(D,{2: one, 31: one}),\
                Vec(D,{2: one, 5: one, 23: one}),\
                Vec(D,{5: one}),\
                Vec(D,{3: one, 2: one, 19: one, 23: one}),\
                Vec(D,{2: one, 3: one, 5: one, 13: one}),\
                Vec(D,{2: one, 3: one, 13: one})])

True

In [81]:
a = 53*771
b = 2*(3**2)*5*13
gcd(a-b, N)

1

In [82]:
a = 52*67*71
b = 2 *(3**2)*5*19*23
gcd(a-b, N)

2419

In [83]:
N/2419 # proper divisor

1.0

In [7]:
## Task 4
def find_a_and_b(v, roots, N):
    '''
    Input: 
     - a {0,1,..., n-1}-vector v over GF(2) where n = len(roots)
     - a list roots of integers
     - an integer N to factor
    Output:
      a pair (a,b) of integers
      such that a*a-b*b is a multiple of N
      (if v is correctly chosen)
    Example:
        >>> roots = [51, 52, 53, 58, 61, 62, 63, 67, 68, 71, 77, 79]
        >>> N = 2419
        >>> v = Vec({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},{1: one, 2: one, 11: one, 5: one})  
        >>> find_a_and_b(v, roots, N)
        (13498888, 778050)
        >>> v = Vec({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},{0: 0, 1: 0, 10: one, 2: one})
        >>> find_a_and_b(v, roots, N)
        (4081, 1170)
    '''
    alist = [roots[i] for i in v.D if v[i] == one]
    a = prod(alist)
    c = prod([x*x-N for x in alist])
    b = intsqrt(c)
    assert b*b == c
    return (a,b)

In [109]:
roots = [51, 52, 53, 58, 61, 62, 63, 67, 68, 71, 77, 79]
N = 2419
v = Vec({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},{1: one, 2: one, 11: one, 5: one})
find_a_and_b(v, roots, N)

(13498888, 778050)

In [110]:
v = Vec({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},{0: 0, 1: 0, 10: one, 2: one})
find_a_and_b(v, roots, N)

(4081, 1170)

In [14]:
%%timeit
# Task 5
N = 2461799993978700679
primelist = primes(1000)
roots, rowlist = find_candidates(N, primelist)
M = echelon.transformation_rows(rowlist, sorted(primelist, reverse=True))

1 loops, best of 3: 8.71 s per loop


In [114]:
v = M[len(M)-1]

In [118]:
a, b = find_a_and_b(v, roots, N) 

In [119]:
gcd(a-b, N)

1230926561

In [15]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %install_default_config  %install_ext  %install_profiles  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%latex  %%